In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Load data into a pandas dataframe
df = pd.read_csv("~/Data/all_subspecialty_geocoded_230315.csv")

In [ ]:
data =df.loc[((df["timepoint"]==2021)|
            (df["timepoint"]==2022)) &
           (df["insurance_cat2"]!="Other") &
            (df["race_ethnic_cat2"]!= "Other") &
            (df["provider_specialty_cat"] == "Mental Health and Behavioral Health") &
            (df["encounter_status_name"]!= "Canceled")]

In [ ]:
#just creating this dataframe to see the number of canceled visits 
data2 =df.loc[((df["timepoint"]==2021)|
            (df["timepoint"]==2022)) &
           (df["insurance_cat2"]!="Other") &
            (df["race_ethnic_cat2"]!= "Other") &
            (df["provider_specialty_cat"] == "Mental Health and Behavioral Health")]

data2["encounter_status_name"].value_counts()

In [ ]:
data["primary_financial_class"].value_counts()

In [ ]:
data

In [ ]:
data["timepoint"].value_counts()

In [ ]:
data["month_quarter"].value_counts()

In [ ]:
data["month_quarter_ord"] = data["month_quarter"].apply(lambda x: 1 if x == "Jan to Mar 2021" 
                                                          else (
                                                          2 if x== "Apr to Jun 2021" else
                                                          (3 if x== "Jul to Sep 2021" else (
                                                          4 if x== "Oct to Dec 2021" else (
                                                          5 if x== "Jan to Mar 2022" else(
                                                          6 if x == "Apr to Jun 2022" else 0))))))
data["month_quarter_ord"].value_counts()

In [ ]:
data["New_Patient"].value_counts()

In [ ]:
data["encounter_status_name"].value_counts()

In [ ]:
data["noshow"] = data["encounter_status_name"].apply(lambda x: 1 if x == "No Show"
                                                          else 0)
data["noshow"].value_counts()

In [ ]:
data["Household Median Income"].describe()

In [ ]:
data["Median_Income_Cat"] = data["Household Median Income"].apply(lambda x: 1 if(x<50000)
                                                                                      else (2 if x<=100000
                                                                                            else (3)))
data["Median_Income_Cat"].value_counts()

In [ ]:
297279/(297279+970774+1523871)

In [ ]:
data["insurance_cat2"].value_counts()

In [ ]:
data['insurance_cat2'].mask(data['insurance_cat2'] == 'Medicare Advantage', "Medicare", inplace=True)
data["insurance_cat2"].value_counts()

In [ ]:
data["age_at_firstVisit"]=data.groupby("ir_id")["age_at_visit"].transform("first")
data[["ir_id","age_at_visit","age_at_firstVisit"]]

In [ ]:
age_cut_labels = ['18-39', '40-64','65+']
age_cut_bins = [0, 40, 65,150]
data['age_firstVisit_cat'] = pd.cut(data['age_at_firstVisit'], bins=age_cut_bins, labels=age_cut_labels)
data['age_firstVisit_cat'].value_counts(dropna=False)

In [ ]:
data["Distance"].describe()

In [ ]:
distance_cut_labels = [1,2,3,4]
distance_cut_bins = [0,3.152,6.137,12.327,350]
data['distance_cat'] = pd.cut(data['Distance'], bins=distance_cut_bins, labels=distance_cut_labels)
data['distance_cat'].value_counts(dropna=False)

In [ ]:
data["Charlson_cat"].value_counts()

In [ ]:
#combining 1-2 as one category
data["Charlson_cat"].mask(data['Charlson_cat'] == '2', "1", inplace=True)
data["Charlson_cat"].value_counts()

In [ ]:
data["Charlson_cat"] = data["Charlson_cat"].apply(lambda x: 0 if x == "0"
                                                          else (1 if x == "1"
                                                          else(2)))
data["Charlson_cat"].value_counts()

# Basic stats for paper

In [ ]:
print(data["ir_id"].nunique())

In [ ]:
data.shape

In [ ]:
data["is_telehealth"].value_counts()

In [ ]:
data["is_telehealth"].value_counts(normalize=True)

In [ ]:
#data.to_csv("~/Data/tableone_data_mh_230813.csv")

In [ ]:
data.columns

In [ ]:
fam = sm.families.Binomial()

In [ ]:
ind = sm.cov_struct.Independence()

In [ ]:
mod8 = smf.gee("noshow ~ distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter)+C(insurance_cat2,Treatment('Commercial')) + C(race_ethnic_cat2,Treatment('Non-Hispanic White'))+ C(is_telehealth,Treatment(False))", "ir_id", data, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
telehealth_dat = data.loc[(data["is_telehealth"]==True)]
inperson_dat = data.loc[(data["is_telehealth"]==False)]

# telehealth

In [ ]:
mod8 = smf.gee("noshow ~ distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter)+C(insurance_cat2,Treatment('Commercial')) + C(race_ethnic_cat2,Treatment('Non-Hispanic White'))", "ir_id", data = telehealth_dat, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
res8.summary()

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

# inperson

In [ ]:
mod8 = smf.gee("noshow ~ distance_cat + C(age_firstVisit_cat) + New_Patient + C(Median_Income_Cat)+ C(Charlson_cat)+C(gender)+C(month_quarter)+C(insurance_cat2,Treatment('Commercial')) + C(race_ethnic_cat2,Treatment('Non-Hispanic White'))", "ir_id", data = inperson_dat, cov_struct=ind, family=fam)


In [ ]:
res8 = mod8.fit()

In [ ]:
print(res8.summary())

In [ ]:
params = res8.params
conf = res8.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

In [ ]:
res8.summary()